In [6]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import chi2
import warnings

# 忽略在模型拟合中可能出现的收敛警告，以保持输出整潔
warnings.filterwarnings('ignore', category=UserWarning)

# ==============================================================================
# Part 1: 数据准备与探索性分析 (EDA)
# ==============================================================================
print("--- [Part 1] 开始数据准备与探索性分析 ---")

# (1) 定义路径
DATA_DIR = '../../Data/0'
RESULT_DIR = 'Result'
os.makedirs(RESULT_DIR, exist_ok=True)
print(f"数据输入路径: {DATA_DIR}")
print(f"结果输出路径: {RESULT_DIR}")

# (2) 加载您已处理好的数据
try:
    processed_file_path = os.path.join(DATA_DIR, '男胎_预处理后数据.csv')
    male_df = pd.read_csv(processed_file_path)
except FileNotFoundError:
    print(f"错误: 未在 '{DATA_DIR}' 目录下找到 '男胎_预处理后数据.csv' 文件。")
    print("请确认您的预处理脚本是否已成功运行，并将生成的 '男胎_预处理后数据.csv' 文件放在了正确的数据目录下。")
    exit()

# (3) 最终分析集清洗
print("根据预处理文件中的标志列进行数据筛选...")
Q1 = male_df['Y染色体浓度'].quantile(0.25)
Q3 = male_df['Y染色体浓度'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
male_df['Y染色体浓度_is_outlier'] = ((male_df['Y染色体浓度'] < lower_bound) | (male_df['Y染色体浓度'] > upper_bound)).astype(int)
print("已补充计算'Y染色体浓度'的离群点标志。")

final_filter = (male_df['Y染色体浓度_is_outlier'] == 0) & \
               (male_df['孕周_is_outlier'] == 0) & \
               (male_df['孕妇BMI_is_outlier'] == 0)

male_df_final_clean = male_df[final_filter].dropna(subset=['Y染色体浓度', '孕周', '孕妇BMI', '孕妇代码']).copy()
print(f"加载数据 {len(male_df)} 行，清洗后最终分析集 {len(male_df_final_clean)} 行。")

# (4) 特征工程
male_df_final_clean['GW_sq'] = male_df_final_clean['孕周'] ** 2
male_df_final_clean['BMI_sq'] = male_df_final_clean['孕妇BMI'] ** 2
male_df_final_clean['GW_x_BMI'] = male_df_final_clean['孕周'] * male_df_final_clean['孕妇BMI']
male_df_final_clean['GW_sq_x_BMI'] = male_df_final_clean['GW_sq'] * male_df_final_clean['孕妇BMI']

# (5) 相关性分析
corr_df = male_df_final_clean[['Y染色体浓度', '孕周', '孕妇BMI']]
corr_table = pd.DataFrame({
    '变量对': ['浓度 vs 孕周', '浓度 vs BMI'],
    '皮尔逊相关系数_r': [corr_df.corr(method='pearson').loc['Y染色体浓度', var] for var in ['孕周', '孕妇BMI']],
    '斯皮尔曼相关系数_rho': [corr_df.corr(method='spearman').loc['Y染色体浓度', var] for var in ['孕周', '孕妇BMI']]
}).round(4)
corr_path = os.path.join(RESULT_DIR, '表格1_相关性分析汇总表.csv')
corr_table.to_csv(corr_path, index=False, encoding='utf-8-sig')
print(f"相关性分析结果已保存至: {corr_path}")
print("--- [Part 1] 数据准备与EDA完成 ---\n")


# ==============================================================================
# Part 2: 核心建模：基准模型构建与M4的系统性优化
# ==============================================================================
print("--- [Part 2] 开始核心建模流程 ---")

# (1) 构建基准模型 M1, M2, M3
print("正在构建基准模型 M1, M2, M3...")
model_data = male_df_final_clean.copy()
m1 = smf.ols('Y染色体浓度 ~ 孕周 + 孕妇BMI', data=model_data).fit()
m2 = smf.ols('Y染色体浓度 ~ 孕周 + GW_sq + 孕妇BMI', data=model_data).fit()
model_data_log = model_data[model_data['Y染色体浓度'] > 0].copy()
m3 = smf.ols('np.log(Y染色体浓度) ~ 孕周 + 孕妇BMI', data=model_data_log).fit()
print("基准模型 M1, M2, M3 构建完成。")

# (2) 最优模型 (M4) 的构建与筛选流程
print("\n开始构建与筛选最优模型 M4...")
base_formula = "Y染色体浓度 ~ 孕周 + 孕妇BMI"
re_formula = "~ 孕周"
candidate_terms = ['GW_sq', 'BMI_sq', 'GW_x_BMI', 'GW_sq_x_BMI']
best_formula = base_formula
best_model = smf.mixedlm(base_formula, data=model_data, groups='孕妇代码', re_formula=re_formula).fit(reml=False, method='cg')
print(f"M4 基准模型拟合完成 (LogLik: {best_model.llf:.2f}, AIC: {best_model.aic:.2f})")
print("-" * 30)
for term in candidate_terms:
    print(f"测试添加项: '{term}'")
    current_formula = f"{best_formula} + {term}"
    current_model = smf.mixedlm(current_formula, data=model_data, groups='孕妇代码', re_formula=re_formula).fit(reml=False, method='cg')
    lr_stat = 2 * (current_model.llf - best_model.llf)
    df_diff = current_model.df_modelwc - best_model.df_modelwc
    p_value = chi2.sf(lr_stat, df_diff)
    print(f"  新模型 LogLik: {current_model.llf:.2f}, AIC: {current_model.aic:.2f}")
    print(f"  似然比检验: LRT Statistic={lr_stat:.3f}, p-value={p_value:.4f}")
    if p_value < 0.05:
        best_model = current_model
        best_formula = current_formula
        print(f"  决策: 接受 '{term}'。模型已更新。")
    else:
        print(f"  决策: 拒绝 '{term}'。模型保持不变。")
    print("-" * 30)
m4_final = best_model
print(f"\n最优模型 M4 筛选完成！最终公式为:\n{best_formula}")
print("--- [Part 2] 核心建模完成 ---\n")


# ==============================================================================
# Part 3: 最终模型评估与结论
# ==============================================================================
print("--- [Part 3] 开始最终模型评估与结果输出 ---")

# (1) 终极对决：四模型性能对比 (增加R²列)
print("正在生成四模型评估对比表...")
models = {'M1_线性回归': m1, 'M2_多项式回归': m2, 'M3_指数回归': m3, 'M4_优化混合效应模型': m4_final}
metrics_list = []
for name, model in models.items():
    r_squared_val = None # 初始化 R²
    if name in ['M1_线性回归', 'M2_多项式回归']:
        y_true = model_data['Y染色体浓度']; y_pred = model.predict(model_data); r_squared_val = model.rsquared_adj
    elif name == 'M3_指数回归':
        y_true = model_data_log['Y染色体浓度']; y_pred = np.exp(model.predict(model_data_log)); pseudo_r2 = np.corrcoef(y_true, y_pred)[0, 1]**2; r_squared_val = f"{pseudo_r2} (伪 R²)"
    elif name == 'M4_优化混合效应模型':
        y_true = model_data['Y染色体浓度']; y_pred = model.predict(model_data)
        var_fe = np.var(model.fittedvalues); random_effects = model.random_effects; var_re_approx = np.var([np.dot(dr, er) for dr, er in zip(model.model.exog_re, random_effects.values())]); var_resid = model.scale
        r2_marginal = var_fe / (var_fe + var_re_approx + var_resid); r2_conditional = (var_fe + var_re_approx) / (var_fe + var_re_approx + var_resid)
        r_squared_val = f"边际:{r2_marginal:.4f}, 条件:{r2_conditional:.4f}"
        r2_table = pd.DataFrame({'指标': ['边际 R-squared', '条件 R-squared'], '值': [r2_marginal, r2_conditional]}).round(4)
        r2_path = os.path.join(RESULT_DIR, '表格5_最终模型M4_R2分解表.csv'); r2_table.to_csv(r2_path, index=False, encoding='utf-8-sig'); print(f"M4 的 R-squared 分解结果已保存至: {r2_path}")
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    metrics_list.append({'模型': name, '对数似然值': model.llf, 'AIC': model.aic, 'BIC': model.bic, 'RMSE': rmse, 'R_squared': r_squared_val})
comparison_table = pd.DataFrame(metrics_list); comparison_table = comparison_table[['模型', '对数似然值', 'AIC', 'BIC', 'RMSE', 'R_squared']]
comp_path = os.path.join(RESULT_DIR, '表格2_四模型评估对比表.csv'); comparison_table.to_csv(comp_path, index=False, encoding='utf-8-sig'); print(f"四模型评估对比表已保存至: {comp_path}")

# (2) 最优模型 (M4) 深度分析
print("正在保存最优模型 M4 的详细结果...")

# --- [核心修正] ---
# 错误原因: .pvalues 和 .tvalues 属性包含所有模型参数（固定+随机）的结果，
# 而 .fe_params 和 .bse_fe 只包含固定效应参数，导致数组长度不一致。
# 解决方案: 使用固定效应参数的索引 (fe_params.index) 来筛选 pvalues 和 tvalues，
# 确保所有用于构建DataFrame的数组长度都相等。

# 提取仅与固定效应相关的参数
fe_params = m4_final.fe_params
fe_bse = m4_final.bse_fe

# 使用 fe_params 的索引来筛选 p值、z值和置信区间，保证长度一致
fe_index = fe_params.index
fe_pvalues = m4_final.pvalues.loc[fe_index]
fe_zvalues = m4_final.tvalues.loc[fe_index]
fe_conf_int = m4_final.conf_int().loc[fe_index]
# --- [修正结束] ---

# 使用提取出的高精度数值构建DataFrame，现在所有列的长度都相同
fe_df = pd.DataFrame({
    '变量': fe_index,
    '系数': fe_params.values,
    '标准误': fe_bse.values,
    'z值': fe_zvalues.values,
    'p值': fe_pvalues.values,
    'CI_lower': fe_conf_int.iloc[:, 0].values,
    'CI_upper': fe_conf_int.iloc[:, 1].values
})

fe_path = os.path.join(RESULT_DIR, '表格3_最终模型M4_固定效应表.csv')
fe_df.to_csv(fe_path, index=False, encoding='utf-8-sig')

# 方差成分表的生成逻辑保持不变
re_names = m4_final.cov_re.index.tolist()
re_variances = np.diag(m4_final.cov_re)
vc_table = pd.DataFrame({
    '方差成分': [f'{name} Var' for name in re_names] + ['Residual Var'],
    '估计值': np.append(re_variances, m4_final.scale)
})
vc_path = os.path.join(RESULT_DIR, '表格4_最终模型M4_方差成分表.csv')
vc_table.to_csv(vc_path, index=False, encoding='utf-8-sig')

# (3) 保存所有模型摘要
print("正在保存所有模型的详细摘要...")
summary_path = os.path.join(RESULT_DIR, 'M1_M2_M3_M4_模型详细摘要.txt')
with open(summary_path, 'w', encoding='utf-8') as f:
    for name, model in models.items():
        f.write("="*40 + f" {name} " + "="*40 + "\n")
        f.write(str(model.summary()) + "\n\n")
print(f"所有模型摘要已保存至: {summary_path}")

# (4) 打印 M4 的高精度系数值 (用于撰写论文公式)
print("\n" + "="*50)
print("--- M4模型高精度系数值 (用于撰写论文公式) ---")
high_precision_params = m4_final.params
print(high_precision_params)
print("="*50 + "\n")

print("\n--- [Part 3] 所有分析与输出已全部完成！---")

--- [Part 1] 开始数据准备与探索性分析 ---
数据输入路径: ../../Data/0
结果输出路径: Result
根据预处理文件中的标志列进行数据筛选...
已补充计算'Y染色体浓度'的离群点标志。
加载数据 998 行，清洗后最终分析集 962 行。
相关性分析结果已保存至: Result\表格1_相关性分析汇总表.csv
--- [Part 1] 数据准备与EDA完成 ---

--- [Part 2] 开始核心建模流程 ---
正在构建基准模型 M1, M2, M3...
基准模型 M1, M2, M3 构建完成。

开始构建与筛选最优模型 M4...
M4 基准模型拟合完成 (LogLik: 2347.28, AIC: -4680.56)
------------------------------
测试添加项: 'GW_sq'
  新模型 LogLik: 2367.64, AIC: -4719.27
  似然比检验: LRT Statistic=40.709, p-value=0.0000
  决策: 接受 'GW_sq'。模型已更新。
------------------------------
测试添加项: 'BMI_sq'
  新模型 LogLik: 2368.79, AIC: -4719.58
  似然比检验: LRT Statistic=2.312, p-value=0.1284
  决策: 拒绝 'BMI_sq'。模型保持不变。
------------------------------
测试添加项: 'GW_x_BMI'
  新模型 LogLik: 2367.84, AIC: -4717.69
  似然比检验: LRT Statistic=0.414, p-value=0.5199
  决策: 拒绝 'GW_x_BMI'。模型保持不变。
------------------------------
测试添加项: 'GW_sq_x_BMI'
  新模型 LogLik: 2367.76, AIC: -4717.51
  似然比检验: LRT Statistic=0.242, p-value=0.6230
  决策: 拒绝 'GW_sq_x_BMI'。模型保持不变。
------------------------------


In [7]:
import pandas as pd
import numpy as np
import os
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

# ==============================================================================
# Part 0: 环境设置与数据准备
# ==============================================================================
# --- 设置绘图风格 (确保中文与字体大小符合要求) ---
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.size'] = 14
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 20


# --- 路径定义 (根据您的规范) ---
DATA_DIR = '../../Data/0'
RESULT_DIR = 'Result'
os.makedirs(RESULT_DIR, exist_ok=True)
print(f"数据输入路径: {DATA_DIR}")
print(f"结果输出路径: {RESULT_DIR}")


# --- 数据加载与清洗 (生成用于所有绘图的数据集) ---
raw_df = pd.read_csv(os.path.join(DATA_DIR, '男胎_预处理后数据.csv'))

# 1. 计算Y染色体浓度的离群点标志
Q1 = raw_df['Y染色体浓度'].quantile(0.25)
Q3 = raw_df['Y染色体浓度'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
raw_df['Y浓度_is_outlier'] = ((raw_df['Y染色体浓度'] < lower_bound) | (raw_df['Y染色体浓度'] > upper_bound)).astype(int)

# 2. 筛选最终用于建模的分析集
final_filter = (raw_df['Y浓度_is_outlier'] == 0) & \
               (raw_df['孕周_is_outlier'] == 0) & \
               (raw_df['孕妇BMI_is_outlier'] == 0)
model_data = raw_df[final_filter].dropna(subset=['Y染色体浓度', '孕周', '孕妇BMI', '孕妇代码']).copy()
model_data['GW_sq'] = model_data['孕周'] ** 2 # 为M4准备二次项

print(f"数据准备完成。原始数据集 {len(raw_df)} 行，最终分析集 {len(model_data)} 行。")


# ==============================================================================
# Part 1: 模型拟合 (仅拟合绘图所需的M1和M4)
# ==============================================================================
# 拟合简单的线性模型 M1 (用于对比图)
m1 = smf.ols('Y染色体浓度 ~ 孕周 + 孕妇BMI', data=model_data).fit()

# 拟合最终的混合效应模型 M4 (用于拟合、残差和核心结果图)
m4_formula = "Y染色体浓度 ~ 孕周 + 孕妇BMI + GW_sq"
m4_final = smf.mixedlm(m4_formula, data=model_data, groups='孕妇代码', re_formula="~ 孕周").fit(reml=False, method='cg')

print("模型 M1 和 M4 已拟合完成，准备开始批量绘图...")


# ==============================================================================
# Part 2: 核心可视化绘图 (生成文档中所有图表)
# ==============================================================================

# --- 图1: Y浓度离群点剔除可视化.png ---
fig, axes = plt.subplots(1, 2, figsize=(16, 8), sharey=True)
palette = {0: '#4B6A94', 1: '#D45F5B'}
# 剔除前
sns.boxplot(y='Y染色体浓度', data=raw_df, ax=axes[0], color='lightgray', width=0.4, fliersize=0)
sns.stripplot(y='Y染色体浓度', x=np.zeros(len(raw_df)), data=raw_df, hue='Y浓度_is_outlier', 
              palette=palette, jitter=0.15, alpha=0.7, ax=axes[0])
axes[0].set_title('离群点剔除前')
axes[0].set_ylabel('Y染色体浓度')
axes[0].set_xlabel('')
axes[0].set_xticks([])
handles, labels = axes[0].get_legend_handles_labels()
axes[0].legend(handles, ['正常值', '离群点'], title='数据点类型')
# 剔除后
sns.boxplot(y=model_data['Y染色体浓度'], ax=axes[1], color='lightgray', width=0.4)
sns.stripplot(y=model_data['Y染色体浓度'], ax=axes[1], color=palette[0], jitter=0.15, alpha=0.7)
axes[1].set_title('离群点剔除后')
axes[1].set_ylabel('')
axes[1].set_xlabel('')
axes[1].set_xticks([])
plt.tight_layout()
sns.despine()
fig1_path = os.path.join(RESULT_DIR, 'Y浓度离群点剔除可视化.png')
plt.savefig(fig1_path, dpi=300, bbox_inches='tight')
print(f"图1: 已保存至 {fig1_path}")
plt.close()


# --- 图2: Y浓度_vs_变量_散点图.png ---
fig, axes = plt.subplots(1, 2, figsize=(18, 8))
# Y浓度 vs 孕周
sns.regplot(x='孕周', y='Y染色体浓度', data=model_data, ax=axes[0],
            scatter_kws={'alpha': 0.3, 'color': '#4B6A94'}, 
            line_kws={'color': '#D45F5B', 'linewidth': 2.5})
axes[0].set_title('Y染色体浓度 vs. 孕周')
axes[0].set_xlabel('孕周 (周)')
axes[0].set_ylabel('Y染色体浓度')
axes[0].grid(True, linestyle='--', alpha=0.6)
# Y浓度 vs 孕妇BMI
sns.regplot(x='孕妇BMI', y='Y染色体浓度', data=model_data, ax=axes[1],
            scatter_kws={'alpha': 0.3, 'color': '#4B6A94'},
            line_kws={'color': '#D45F5B', 'linewidth': 2.5})
axes[1].set_title('Y染色体浓度 vs. 孕妇BMI')
axes[1].set_xlabel('孕妇BMI')
axes[1].set_ylabel('')
axes[1].grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
sns.despine()
fig2_path = os.path.join(RESULT_DIR, 'Y浓度_vs_变量_散点图.png')
plt.savefig(fig2_path, dpi=300, bbox_inches='tight')
print(f"图2: 已保存至 {fig2_path}")
plt.close()


# --- 图3: 模型拟合效果对比图.png ---
plot_preds = pd.DataFrame({'孕周': np.linspace(model_data['孕周'].min(), model_data['孕周'].max(), 200)})
plot_preds['孕妇BMI'] = model_data['孕妇BMI'].mean()
plot_preds['GW_sq'] = plot_preds['孕周']**2
plot_preds['M1_pred'] = m1.predict(plot_preds)
plot_preds['M4_pred'] = m4_final.predict(plot_preds)
plt.figure(figsize=(12, 8))
sns.scatterplot(x='孕周', y='Y染色体浓度', data=model_data, alpha=0.3, color='#4B6A94', label='观测数据点')
plt.plot(plot_preds['孕周'], plot_preds['M1_pred'], color='#D45F5B', linewidth=3, linestyle='--', label='M1: 线性模型拟合')
plt.plot(plot_preds['孕周'], plot_preds['M4_pred'], color='#6AAB9C', linewidth=3.5, linestyle='-', label='M4: 最终模型拟合 (U型趋势)')
plt.xlabel('孕周 (周)')
plt.ylabel('Y染色体浓度')
plt.legend(loc='best')
plt.grid(True, linestyle='--', alpha=0.6)
sns.despine()
fig3_path = os.path.join(RESULT_DIR, '模型拟合效果对比图.png')
plt.savefig(fig3_path, dpi=300, bbox_inches='tight')
print(f"图3: 已保存至 {fig3_path}")
plt.close()


# --- 图4: 最终模型M4_可视化.png (核心结果图) ---
plt.figure(figsize=(14, 9))
# 1. 背景散点
plt.scatter(model_data['孕周'], model_data['Y染色体浓度'], alpha=0.15, color='gray', label='观测数据')
# 2. 个体趋势线 (随机抽取25位)
unique_subjects = model_data['孕妇代码'].unique()
sample_size = min(25, len(unique_subjects))
sample_subjects = np.random.choice(unique_subjects, sample_size, replace=False)
for i, subject in enumerate(sample_subjects):
    subject_data = model_data[model_data['孕妇代码'] == subject].sort_values('孕周')
    subject_preds = m4_final.predict(subject_data)
    label = '个体趋势 (随机样本)' if i == 0 else None
    plt.plot(subject_data['孕周'], subject_preds, color='#4B6A94', linestyle='--', linewidth=1, alpha=0.7, label=label)
# 3. 总体平均趋势线
fixed_effect_preds = m4_final.predict(plot_preds)
plt.plot(plot_preds['孕周'], fixed_effect_preds, color='#D45F5B', linewidth=4, label='总体平均趋势 (固定效应)')
plt.xlabel('孕周 (周)')
plt.ylabel('Y染色体浓度')
plt.legend(loc='best')
plt.grid(True, linestyle='--', alpha=0.6)
sns.despine()
fig4_path = os.path.join(RESULT_DIR, '最终模型M4_可视化.png')
plt.savefig(fig4_path, dpi=300, bbox_inches='tight')
print(f"图4: 已保存至 {fig4_path}")
plt.close()


# --- 图5: 最终模型M4_残差分析图.png ---
residuals = m4_final.resid
fitted_values = m4_final.fittedvalues
plt.figure(figsize=(12, 8))
sns.scatterplot(x=fitted_values, y=residuals, alpha=0.5, color='#4B6A94')
plt.axhline(y=0, color='#D45F5B', linestyle='--', linewidth=2)
plt.xlabel('拟合值')
plt.ylabel('残差')
plt.grid(True, linestyle='--', alpha=0.6)
sns.despine()
fig5_path = os.path.join(RESULT_DIR, '最终模型M4_残差分析图.png')
plt.savefig(fig5_path, dpi=300, bbox_inches='tight')
print(f"图5: 已保存至 {fig5_path}")
plt.close()

print("\n--- 所有可视化图表均已生成完毕！---")

数据输入路径: ../../Data/0
结果输出路径: Result
数据准备完成。原始数据集 998 行，最终分析集 962 行。
模型 M1 和 M4 已拟合完成，准备开始批量绘图...
图1: 已保存至 Result\Y浓度离群点剔除可视化.png
图2: 已保存至 Result\Y浓度_vs_变量_散点图.png
图3: 已保存至 Result\模型拟合效果对比图.png
图4: 已保存至 Result\最终模型M4_可视化.png
图5: 已保存至 Result\最终模型M4_残差分析图.png

--- 所有可视化图表均已生成完毕！---
